In [10]:
import re
import os
import json # Importar el módulo JSON

from bs4 import BeautifulSoup

In [11]:
def read_html_from_file(file_path: str) -> str | None:
    """
    Lee el contenido de un archivo HTML desde una ruta específica.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo '{file_path}': {e}")
        return None

def extract_project_data(row, pi_name, last_updated):
    """
    Extrae la información detallada de una única fila, incluyendo
    datos bibliográficos como DOI, PMID y título.
    """
    try:
        cells = row.find_all('td')
        
        # --- Datos del Proyecto ---
        project_cell = cells[1]
        project_link_tag = project_cell.find('a', href=re.compile("TASKID"))
        project_name = project_link_tag.get_text(strip=True)
        relative_link = project_link_tag['href']
        project_link = f"https://taskbook.nasaprs.com/tbp/{relative_link}"
        task_id_match = re.search(r'TASKID=(\d+)', relative_link)
        task_id = task_id_match.group(1) if task_id_match else "No encontrado"

        # --- Datos Bibliográficos (Celda más compleja) ---
        bibliography_cell = cells[2]
        cell_text = bibliography_cell.get_text(separator=' ', strip=True)

        # Título del Artículo (texto entre comillas)
        title_match = re.search(r'"([^"]+)"', cell_text)
        article_title = title_match.group(1).strip() if title_match else "No encontrado"

        # Autores (texto antes de la primera comilla)
        authors = cell_text.split('"')[0].strip()

        # Inicializar campos de enlaces e IDs
        doi_link = "No encontrado"
        pubmed_link = "No encontrado"
        pubmed_id = "No encontrado"
        pmc_link = "No encontrado"
        pmc_id = "No encontrado"

        # Buscar todos los enlaces <a> en la celda para extraer sus datos
        links_in_cell = bibliography_cell.find_all('a')
        for link in links_in_cell:
            href = link.get('href', '')
            text = link.get_text(strip=True)
            
            if "doi.org" in href:
                doi_link = href
            elif "db=pubmed" in href:
                pubmed_link = href
                if "PMID:" in text:
                    pubmed_id = text.replace("PMID:", "").strip()
            elif "pmc/articles" in href:
                pmc_link = href
                if "PMCID:" in text:
                    pmc_id = text.replace("PMCID:", "").strip()

        return {
            'PI Name': pi_name,
            'Bibliography Last Updated': last_updated,
            'Project Name': project_name,
            'Project Link': project_link,
            'Task ID': task_id,
            'Authors': authors,
            'Article Title': article_title,
            'DOI': doi_link,
            'PubMed ID': pubmed_id,
            'PubMed Link': pubmed_link,
            'PubMed Central ID': pmc_id,
            'PubMed Central Link': pmc_link
        }
    except (AttributeError, IndexError):
        return None
    
def parse_bibliography_results(html_content: str) -> list[dict]:
    """
    Parsea el contenido HTML de un ÚNICO archivo.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    results_table = soup.find('table', class_='intro')
    
    if not results_table:
        return []

    all_projects = []
    current_pi_name = None
    current_last_updated = None

    for row in results_table.find_all('tr'):
        if 'title' in row.get('class', []):
            cells = row.find_all('td')
            if len(cells) >= 5:
                current_pi_name = cells[2].get_text(strip=True)
                current_last_updated = cells[4].get_text(strip=True)
        
        elif row.find('a', href=re.compile("TASKID")):
            project_data = extract_project_data(row, current_pi_name, current_last_updated)
            if project_data:
                all_projects.append(project_data)
                
    return all_projects

def process_html_folder(folder_path: str) -> list[dict]:
    """
    Procesa todos los archivos HTML en una carpeta y agrega los resultados.
    """
    if not os.path.isdir(folder_path):
        print(f"Error: La ruta '{folder_path}' no es un directorio válido.")
        return []
        
    aggregated_results = []
    print(f"Escaneando directorio: {folder_path}")

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.html', '.htm')):
            full_path = os.path.join(folder_path, filename)
            print(f"  -> Procesando archivo: {filename}")
            
            html_content = read_html_from_file(full_path)
            if html_content:
                projects_from_file = parse_bibliography_results(html_content)
                for project in projects_from_file:
                    project['Source File'] = filename
                
                aggregated_results.extend(projects_from_file)

    return aggregated_results

def save_results_to_json(project_list: list[dict], output_path: str):
    """
    Guarda una lista de diccionarios en un archivo JSON.

    Args:
        project_list: La lista de datos de proyectos.
        output_path: La ruta del archivo JSON de salida.
    """
    if not project_list:
        print("No hay datos para guardar en el archivo JSON.")
        return
        
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            # json.dump escribe la lista en el archivo
            # indent=4 formatea el archivo para que sea legible
            # ensure_ascii=False permite guardar caracteres especiales como acentos
            json.dump(project_list, f, indent=4, ensure_ascii=False)
        print(f"\n¡Éxito! Los resultados se han guardado en: {output_path}")
    except Exception as e:
        print(f"\nError: No se pudo guardar el archivo JSON. Razón: {e}")

def display_results(project_list: list[dict]):
    """
    Imprime los resultados extraídos en la consola.
    """
    if not project_list:
        print("No se extrajeron datos para mostrar.")
        return

    print(f"\nTotal de registros extraídos: {len(project_list)}")


In [12]:

# --- Bloque principal de ejecución ---
if __name__ == "__main__":
    # --- MODIFICA ESTA LÍNEA ---
    # Pon la ruta a la CARPETA que contiene tus archivos HTML.
    folder_path = "taskbook/table_html"
    output_json_path = "taskbook/nasa_tasks.json"
    
    # 1. Procesar toda la carpeta de archivos HTML
    all_extracted_projects = process_html_folder(folder_path)
    
    # 2. Guardar los resultados combinados en un archivo JSON
    save_results_to_json(all_extracted_projects, output_json_path)
    
    # 3. Mostrar una vista previa de los resultados en la consola
    display_results(all_extracted_projects)

Escaneando directorio: taskbook/table_html
  -> Procesando archivo: 1.html
  -> Procesando archivo: 10.html
  -> Procesando archivo: 11.html
  -> Procesando archivo: 12.html
  -> Procesando archivo: 2.html
  -> Procesando archivo: 3.html
  -> Procesando archivo: 4.html
  -> Procesando archivo: 5.html
  -> Procesando archivo: 6.html
  -> Procesando archivo: 7.html
  -> Procesando archivo: 8.html
  -> Procesando archivo: 9.html

¡Éxito! Los resultados se han guardado en: taskbook/nasa_tasks.json

Total de registros extraídos: 557
